In [22]:
import numpy as np
import pandas as pd
import pytest
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
from sktime.datasets import load_airline
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.theta import ThetaForecaster
from sktime.performance_metrics.forecasting.probabilistic import PinballLoss, EmpiricalCoverage, ConstraintViolation

list_of_metrics = [PinballLoss]

# test data
y = np.log1p(load_airline())
y_train, y_test = temporal_train_test_split(y)
fh = np.arange(len(y_test)) + 1
f = ThetaForecaster(sp=12)
f.fit(y_train)


QUANTILE_PRED = f.predict_quantiles(fh=fh, alpha=[0.5, 0.9])
INTERVAL_PRED = f.predict_interval(fh=fh, coverage=[0.9, 0.8])

In [44]:
QUANTILE_PRED.columns.set_levels(["V1"], level=0, inplace=True)
QUANTILE_PRED2 = QUANTILE_PRED.copy()
QUANTILE_PRED2.columns.set_levels(["V2"], level=0, inplace=True)
multi_df = pd.concat([QUANTILE_PRED, QUANTILE_PRED2], axis = 1)

In [62]:
loss = PinballLoss(multioutput="raw_values", score_average=False)
loss(np.column_stack((y_test, y_test)), multi_df)

,0.5,0.9
0,0.026143,0.015572


In [67]:
def _handle_multioutput(loss, multioutput):
        """Specificies how multivariate outputs should be handled.

        Parameters
        ----------
        loss : float, np.ndarray the evaluated metric value.

        multioutput : string "uniform_average" or "raw_values" determines how \
            multioutput results will be treated.
        """
        if isinstance(multioutput, str):
            if multioutput == "raw_values":
                return loss
            elif multioutput == "uniform_average":
                # pass None as weights to np.average: uniform mean
                multioutput = None
            else:
                raise ValueError(
                    "multioutput is expected to be 'raw_values' "
                    "or 'uniform_average' but we got %r"
                    " instead." % multioutput
                )

        if loss.ndim > 1:
            out = np.average(loss, weights=multioutput, axis=1)
        else:
            out = np.average(loss, weights=multioutput)
        return out


In [99]:
a_mat = np.repeat((y_pred.columns.get_level_values(1).to_numpy().reshape(1,-1)), repeats = 5, axis = 0)

In [111]:
a = (pd.DataFrame(a_mat, columns=y_pred.columns))

In [142]:
a.mean(axis = 1, level = 0)

,V1,V2
0,0.7,0.7
1,0.7,0.7
2,0.7,0.7
3,0.7,0.7
4,0.7,0.7


In [147]:
np.repeat(y_true, 2, axis = 1)

array([[5.83188248, 5.83188248, 5.83188248, 5.83188248],
       [5.7651911 , 5.7651911 , 5.7651911 , 5.7651911 ],
       [5.89440283, 5.89440283, 5.89440283, 5.89440283],
       [5.85507192, 5.85507192, 5.85507192, 5.85507192],
       [5.89715387, 5.89715387, 5.89715387, 5.89715387],
       [6.07764224, 6.07764224, 6.07764224, 6.07764224],
       [6.19847872, 6.19847872, 6.19847872, 6.19847872],
       [6.22653667, 6.22653667, 6.22653667, 6.22653667],
       [6.00388707, 6.00388707, 6.00388707, 6.00388707],
       [5.88610403, 5.88610403, 5.88610403, 5.88610403],
       [5.73979291, 5.73979291, 5.73979291, 5.73979291],
       [5.8230459 , 5.8230459 , 5.8230459 , 5.8230459 ],
       [5.88887796, 5.88887796, 5.88887796, 5.88887796],
       [5.83773045, 5.83773045, 5.83773045, 5.83773045],
       [6.00881319, 6.00881319, 6.00881319, 6.00881319],
       [5.98393628, 5.98393628, 5.98393628, 5.98393628],
       [6.04263283, 6.04263283, 6.04263283, 6.04263283],
       [6.15909539, 6.15909539,

In [154]:
y_true = np.column_stack((y_test, y_test))
y_pred = multi_df
alphas = [0.5, 0.9]
multioutput = "raw_values"
alpha_mat = np.repeat(
    y_pred.columns.get_level_values(1).to_numpy().reshape(1,-1),
    repeats = y_pred.shape[0],
    axis = 0
    )

diff = np.repeat(y_true, 2, axis = 1) - y_pred.to_numpy()
sign = (diff >= 0).astype(diff.dtype)
loss = alpha_mat * sign * diff - (1 - alpha_mat) * (1 - sign) * diff

out_df = pd.DataFrame(loss, columns=y_pred.columns)
out_df.mean(axis=1).mean()

0.02085756704835874

In [58]:
loss(y_test, QUANTILE_PRED)

0.020857567048358745

In [8]:
QUANTILE_PRED

Quantiles          
              0.5       0.9
1958-01  5.847790  5.907319
1958-02  5.841117  5.911932
1958-03  5.998219  6.078754
1958-04  5.954095  6.043296
1958-05  5.950747  6.047845
1958-06  6.087700  6.192099
1958-07  6.192112  6.303334
1958-08  6.184265  6.301914
1958-09  6.056055  6.179799
1958-10  5.904250  6.033802
1958-11  5.752167  5.887277
1958-12  5.897841  6.038289
1959-01  5.913132  6.058724
1959-02  5.906324  6.056883
1959-03  6.065117  6.220485
1959-04  6.020439  6.180471
1959-05  6.016992  6.181556
1959-06  6.155407  6.324381
1959-07  6.260917  6.434190
1959-08  6.252919  6.430386
1959-09  6.123224  6.304788
1959-10  5.969675  6.155246
1959-11  5.815847  6.005341
1959-12  5.963074  6.156410
1960-01  5.978474  6.175578
1960-02  5.971531  6.172332
1960-03  6.132015  6.336447
1960-04  6.086783  6.294782
1960-05  6.083238  6.294743
1960-06  6.223114  6.438069
1960-07  6.329722  6.548071
1960-08  6.321573  6.543266
1960-09  6.190393  6.415379
1960-10  6.035099  6.263332
1960-11  5.879528  6.110961
1960-12  6.028307  6.262896

In [2]:
loss = PinballLoss()


In [3]:
loss(y_test, QUANTILE_PRED)

0.020857567048358745

In [11]:
loss.evaluate(y_true = y_test, y_pred = QUANTILE_PRED)

,0.5
0,0.026143


In [1]:
# -*- coding: utf-8 -*-
"""Tests for probabilistic quantiles."""
import numpy as np
import pandas as pd
import pytest

from sktime.datasets import load_airline
from sktime.utils._testing.series import _make_series
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.compose import ColumnEnsembleForecaster
from sktime.performance_metrics.forecasting.probabilistic import PinballLoss, EmpiricalCoverage, ConstraintViolation

list_of_metrics = [PinballLoss, EmpiricalCoverage]

# test data
#y = np.log1p(load_airline())
y = _make_series(n_columns=1)
y_train, y_test = temporal_train_test_split(y)
fh = np.arange(len(y_test)) + 1
f = ColumnEnsembleForecaster(ThetaForecaster(sp=12))
f.fit(y_train)

c:\users\e.enticott\repositories\sktime\sktime\sktime\datatypes\_series\_check.py:43: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.Int64Index, pd.RangeIndex, pd.PeriodIndex, pd.DatetimeIndex)
c:\users\e.enticott\repositories\sktime\sktime\sktime\datatypes\_hierarchical\_check.py:50: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.Int64Index, pd.RangeIndex, pd.PeriodIndex, pd.DatetimeIndex)
c:\users\e.enticott\repositories\sktime\sktime\sktime\datatypes\_hierarchical\_check.py:51: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_MULTIINDEX_TYPES = (pd.Int64Index, pd.RangeIndex)
c:\users\e.enticott\repositories\sktime

ColumnEnsembleForecaster(forecasters=ThetaForecaster(sp=12))

In [2]:
f.predict_quantiles(fh)

NotImplementedError: ColumnEnsembleForecaster does not have the capability to return quantile predictions. If you think this estimator should have the capability, please open an issue on sktime.